## Asumption for making model



I am making character similarity based on the dialogues they spoken in the series,because mostly character of Human assemble with their talking way.Therefore I am using same for predicting similar character in the Games of Thrones Serial 

In [24]:
#Importing Libraries
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings("ignore")

### Read a datatset

In [2]:
df = pd.read_json('script-bag-of-words.json')

In [3]:
df.tail(5)

,episodeAlt,seasonNum,episodeNum,episodeTitle,text
68,S8E2,8,2,A Knight of the Seven Kingdoms,"[{'name': 'Daenerys Targaryen', 'text': 'About..."
69,S8E3,8,3,The Long Night,"[{'name': 'Northman #1', 'text': 'Oi!'}, {'nam..."
70,S8E4,8,4,The Last of the Starks,"[{'name': 'Jon Snow', 'text': 'And Everyone It..."
71,S8E5,8,5,The Bells,"[{'name': 'Lord Varys', 'text': 'And? Come Not..."
72,S8E6,8,6,The Iron Throne,"[{'name': 'Tyrion Lannister', 'text': 'I'll fi..."


In [4]:
#Dialogues in last episode 
df.iloc[-1]['text']

[{'name': 'Tyrion Lannister', 'text': "I'll find later. you"},
 {'name': 'Jon Snow', 'text': "It's Let me men not safe. send some with you."},
 {'name': 'Tyrion Lannister', 'text': "I'm alone. going"},
 {'name': 'Grey Worm',
  'text': 'Daenerys I In Queen, Targaryen, die. name of one sentence the the to true you'},
 {'name': 'Jon Snow',
  'text': "Grey It's These Worm! are men over. prisoners."},
 {'name': 'Grey Worm',
  'text': "It Queen's are defeated. enemies is not over the until"},
 {'name': 'Davos Seaworth',
  'text': "How They're be? defeated do knees. more much on their them to want you"},
 {'name': 'Grey Worm', 'text': 'They are breathing.'},
 {'name': 'Davos Seaworth', 'text': 'Look We around friend. won. you,'},
 {'name': 'Grey Worm', 'text': "I commands, my not obey queen's yours."},
 {'name': 'Jon Snow', 'text': "And Queen's are commands? the what"},
 {'name': 'Grey Worm',
  'text': '"Kill Cersei Lannister." These They all are chose fight follow for free her. men. to who'}

### How to extract dialogue of every character

In [5]:
dialogue={}
for index,row in df.iterrows():
    for item in row['text']:
        if item['name'] in dialogue:
            dialogue[item['name']]=dialogue[item['name']] + item['text']
        else:
            dialogue[item['name']] = item['text'] + " "

In [6]:
len(dialogue)

817

<font color='blue'> It is the number of character in the shows </font>

In [7]:
df_got = pd.DataFrame()
df_got['character'] = dialogue.keys()
df_got['words'] = dialogue.values()

In [8]:
df_got.head(6)

,character,words
0,Will,"Easy, boy. I've I've Wildlings a a do ever in ..."
1,Waymar Royce,One They're What a and another before d'you ea...
2,Gared,Wall. We back head should the to Our They We W...
3,Jon Snow,Father's Go on. watching. And mother. yourBran...
4,Septa Mordane,"Fine Well always. as done. work, I Quite beaut..."
5,Sansa Stark,Thank you. Will shut up? youWould please shut ...


In [9]:
df_got['num_words'] = df_got['words'].apply(lambda x:len(x.split()))

In [10]:
df_got = df_got.sort_values('num_words',ascending=False)

In [207]:
df_got.head(10)

,character,words,num_words
17,Tyrion Lannister,It Mmh. Northern about girls. is say the they ...,25924
13,Cersei Lannister,And And Casterly One Rock. When about afraid. ...,14294
3,Jon Snow,Father's Go on. watching. And mother. yourBran...,11488
20,Daenerys Targaryen,We've a and anything. asked been for for guest...,11202
12,Jaime Lannister,"As I It's brother, duty feel it's much. my sho...",10823
5,Sansa Stark,Thank you. Will shut up? youWould please shut ...,8119
42,Petyr Baelish,But Catelyn I've Kingdom Lady Lord No Stark. a...,7940
40,Lord Varys,Lord Stark. I Joffrey's Kingsroad. Prince We a...,7012
147,Davos Seaworth,Not now. Stannis We don't even follow he if is...,6241
59,Samwell Tarly,"Hill. Hill. Horn Horn I I I've Samwell Tarly, ...",6192


In [37]:
df_got.to_pickle("df.pkl")

<font color='red'>This show who have more dialogues in descending order</font>

In [196]:
#taking only 100 character who speaks minimum 500 words

new_df = df_got.head(100)

## Feature Extraction

In [197]:
cv_got = CountVectorizer(stop_words='english')

In [198]:
embeddings =  cv_got.fit_transform(new_df['words']).toarray()

In [199]:
embeddings.shape

(100, 15335)

In [200]:
embeddings = embeddings.astype('float64')

## Bag of words

In [213]:
tf_got = TfidfVectorizer()

In [214]:
emb = tf_got.fit_transform(new_df['words']).toarray()

In [215]:
emb.shape

(100, 15609)

In [216]:
emb = emb.astype('float64')

## Visualization of High Dimension data

In [217]:
tsne = TSNE(n_components=2,verbose=1,random_state=123)
z = tsne.fit_transform(emb)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 100 samples in 0.005s...
[t-SNE] Computed neighbors for 100 samples in 0.021s...
[t-SNE] Computed conditional probabilities for sample 100 / 100
[t-SNE] Mean sigma: 0.283963
[t-SNE] KL divergence after 250 iterations with early exaggeration: 110.169853
[t-SNE] KL divergence after 1000 iterations: 0.930020


In [218]:
z.shape

(100, 2)

In [219]:
# 100 character 2d representation points
new_df['x'] = z.T[0]
new_df['y'] = z.T[1]

In [220]:
new_df

,character,words,num_words,x,y
17,Tyrion Lannister,It Mmh. Northern about girls. is say the they ...,25924,-22.684040,5.098839
13,Cersei Lannister,And And Casterly One Rock. When about afraid. ...,14294,-11.244421,-7.497992
3,Jon Snow,Father's Go on. watching. And mother. yourBran...,11488,5.034934,28.501829
20,Daenerys Targaryen,We've a and anything. asked been for for guest...,11202,-14.843207,23.596085
12,Jaime Lannister,"As I It's brother, duty feel it's much. my sho...",10823,-0.000193,7.776134
...,...,...,...,...,...
132,Rickard Karstark,Castle I'll King North! Red The They and can c...,466,-197.104706,-18.982483
57,Syrio Forel,"Tomorrow You are at be boy. here late, midday....",462,88.333611,-140.602249
112,Kevan Lannister,Tyrion. Catelyn Golden Jaime Lords River River...,460,-73.571671,-66.815109
365,Mace Tyrell,"From Grace, House Margaery May Reach, Tyrell Y...",450,-104.232986,-86.484550


In [221]:
fig = px.scatter(new_df,x='x',y='y',color='character')
fig.show()

In [222]:
new_df.to_pickle("data1.pkl")#TFIDF

In [212]:
new_df.to_pickle("data.pkl")#CV